### Description of Computer firm DB

The database scheme consists of four tables:
- `Product` (maker, model, type)
- `PC` (code, model, speed, ram, hd, cd, price)
- `Laptop` (code, model, speed, ram, hd, screen, price)
- `Printer` (code, model, color, type, price)

The `Product` table contains data on the maker, model number, and type of product ('PC', 'Laptop', or 'Printer'). It is assumed that model numbers in the Product table are unique for all makers and product types.
 
Each personal computer in the `PC table` is unambiguously identified by a unique code, and is additionally characterized by its model (foreign key referring to the Product table), processor speed (in MHz) – speed field, RAM capacity (in Mb) - ram, hard disk drive capacity (in Gb) – hd, CD-ROM speed (e.g, '4x') - cd, and its price.

The `Laptop` table is similar to the PC table, except that instead of the CD-ROM speed, it contains the screen size (in inches) – screen. 

For each printer model in the `Printer table`, its output type (‘y’ for color and ‘n’ for monochrome) – color field, printing technology ('Laser', 'Jet', or 'Matrix') – type, and price are specified.

![Сomputers](computers.gif)

### 4. Find all records from the Printer table containing data about color printers.

In [ ]:
SELECT *
FROM Printer
WHERE color = 'y';

### 5. Find the model number, speed and hard drive capacity of PCs cheaper than $600 having a 12x or a 24x CD drive.

In [ ]:
SELECT model, speed, hd
FROM pc
WHERE price < 600 AND (cd = '12x' OR cd = '24x');

### 6. For each maker producing laptops with a hard drive capacity of 10 Gb or higher, find the speed of such laptops. Result set: maker, speed.

In [ ]:
SELECT DISTINCT p1.maker,l1.speed
FROM product AS p1
JOIN laptop AS l1
ON p1.model = l1.model
WHERE l1.hd >= 10;

### 7. Get the models and prices for all commercially available products (of any type) produced by maker B.

In [ ]:
SELECT all_products.model, all_products.price
FROM (SELECT model, price FROM pc
      UNION
      SELECT model, price FROM laptop
      UNION
      SELECT model, price FROM printer) AS all_products
JOIN product
ON all_products.model = product.model
WHERE product.maker = 'B';

### 8. Find the makers producing PCs but not laptops.

In [ ]:
SELECT DISTINCT maker
FROM product
WHERE type IN ('pc', 'laptop')
EXCEPT
SELECT maker
FROM product
WHERE type = 'laptop';

### 9. Find the makers of PCs with a processor speed of 450 MHz or more. Result set: maker.

In [ ]:
SELECT DISTINCT p1.maker
FROM product AS p1
JOIN pc AS pc1
ON p1.model = pc1.model
WHERE pc1.speed >= 450;